# Fiat-Shamir and Schwartz-Zippel for Faster Emulated Pairings

### 1. Introduction

In the write-up [Faster Extension Field multiplications for Emulated Pairing Circuits](https://hackmd.io/@feltroidprime/B1eyHHXNT#fn3), Feltroid Prime discusses using Schwartz Zippel lemma and Fiat Shamir heuristics to optimize Field multiplications (Sections 1, 2 and 3).

Starting with representing $𝔽_{p12}$ (and equivalently $𝔽_{p6}$) as direct extensions in section 1. Then representing multiplication operation as a polynomial in section 2 (equation 1). Finally using Fiat-Shamir and the Schwartz-Zippel lemma in the circuit to verify the computation.

> ${A(x)*B(x) = Q(x)*P(x) + R(x)}$ &emsp; &emsp; &emsp;${(1)}$

We will expand on this to apply it to larger computations for greater cost savings.

### 2. Challenges

As we try to prove larger computations, say exponentiation of an $𝔽_{p12}$ by n, the degree of ${Q(x)}$ grows linearly with n.
With a large degree comes increased calldata size and computation costs for hashing each coefficient for Fiat-Shamir. To make things easier we prefer an optimised version for larger computations (equation 2).

> $A(x)*B(x) = R(x)$ $mod$ $P(x)$ &emsp; &emsp; &emsp;$(1)$

With ${P(x)}$ for modulo, bitsize of `x` from Fiat Shamir needs to be reduced to $1/(deg P(x))$ of maximum bitsize possible for a modulo operation.

### 3. Introduction
